In [5]:
import cv2
import numpy as np

# Cargar la imagen principal (donde se busca a Wally) y la imagen de plantilla (Wally)
img_scene = cv2.imread('C:/Projects/ITM/INTELIGENCIA ARTIFICIAL/2_Proyecto_Wally/View/img.jpg', cv2.IMREAD_COLOR)
img_wally = cv2.imread('C:/Projects/ITM/INTELIGENCIA ARTIFICIAL/2_Proyecto_Wally/Img/Wally/Wally12.png', cv2.IMREAD_COLOR)

# Convertir a escala de grises
gray_scene = cv2.cvtColor(img_scene, cv2.COLOR_BGR2GRAY)
gray_wally = cv2.cvtColor(img_wally, cv2.COLOR_BGR2GRAY)

# Crear el detector ORB
orb = cv2.ORB_create()

# Encontrar los keypoints y descriptores con ORB
kp1, des1 = orb.detectAndCompute(gray_wally, None)
kp2, des2 = orb.detectAndCompute(gray_scene, None)

# Crear el objeto BFMatcher
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Realizar la coincidencia de descriptores
matches = bf.match(des1, des2)

# Ordenar las coincidencias por su distancia (las mejores coincidencias primero)
matches = sorted(matches, key=lambda x: x.distance)

# Dibujar las primeras 10 coincidencias
img_matches = cv2.drawMatches(img_wally, kp1, img_scene, kp2, matches[:10], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

# Mostrar las coincidencias
cv2.imshow('Matches', img_matches)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Filtrar las mejores coincidencias
good_matches = matches[:10]

# Extraer las ubicaciones de los keypoints en la imagen de Wally y la imagen de la escena
points_wally = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
points_scene = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

# Calcular la matriz de homografía
M, mask = cv2.findHomography(points_wally, points_scene, cv2.RANSAC, 5.0)

# Obtener las dimensiones de la imagen de Wally
h, w = gray_wally.shape

# Definir los puntos de los bordes de la imagen de Wally
pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)

# Transformar los puntos para que correspondan a los de la escena
dst = cv2.perspectiveTransform(pts, M)

# Dibujar el rectángulo en la escena
img_scene = cv2.polylines(img_scene, [np.int32(dst)], True, (0, 255, 0), 3, cv2.LINE_AA)

# Mostrar el resultado final
cv2.imshow('Detected Wally', img_scene)
cv2.waitKey(0)
cv2.destroyAllWindows()
